<h1>Preparation of the German Twitter Data</h1>

<p>This file is for preparing the german tweets. The preparation is mainly for training my own model for sentiment detection with Flair. </p>

In [1]:
import csv
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from HanTa import HanoverTagger as ht

<h1>Einlesen und bestimmung der Felder</h1>
<p>Durch die weitern Operatoren, kann ich die einzelnen Felder in DF bestimmen. Durch die Lambda Funktion werden die Str zu Listen, was die spätere verarbeitung erleichtert. 
    Durch die Parse_dates Funktion wird das Datumsfeld zu einem Timestamp, was die weiter Verarbeitung wesentlich leichter macht.</p>

In [2]:
df_manager = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Raw_File/Manager_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_manager = df_manager.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)

In [3]:
print(len(df_manager))

14451


<h3>Löschen von unnötigen Informationen</h3>
<p>Hier werden Informationen aus den Tweets Texten, die nicht weiter benötigt werden. Bspw. Links</p>

In [4]:
for each in range(len(df_manager)):
    df_manager.loc[df_manager["text"] == 
                      df_manager.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$|\r]', '', 
                                                                                       df_manager.loc[each]["text"]))))

In [5]:
len(df_manager)

14451

<h3>Nicht Kontextbezogene Tweets löschen</h3>
<p>So kann ich prüfen ob bestimmte Begriffe in den Texten enthalten sind und mir die conversation_id ausgeben lassen. Diese lasse ich dann löschen und auch zugehörige Tweets, da diese nicht in den Kontext passen.<br>Die wörter in der Liste müssen klein geschrieben werden, da auch alle Buchstaben für die Prüfung klein geschrieben werden müssen, da sinst nicht alles erkannt wird.</p>
<p>Zu löschende Begriffe</p>
<ul>
    <li>BVB</li>
    <li>Fußball</li>
    <li>Fussball</li>
    <li>Dortmund</li>
    <li>kicker</li>
    <li>fpö</li>
    <li>fpoe</li>
</ul>

In [6]:
not_used_words_list = ["bvb", "fußball", "fussball", "dortmund", "fpö", "fpoe", "kicker"]
tweet_id_without_context_list = []
for each in range(len(df_manager)):
    for word in not_used_words_list:
        if word in df_manager.iloc[each]["text"].lower():
            if df_manager.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_manager.iloc[each]["tweet_id"])

<p>Iteration durch die Liste mit zu löschenden Tweets, bis der Wert zu Beginn und zum Ende gleich ist und es keine
weiteren Tweets mehr gibt, die gefiltert werden müssen. </p>

In [7]:
def check_for_further_tweets():
    start_len = len(tweet_id_without_context_list)
    for each in range(len(tweet_id_without_context_list)):
        df_tweets = df_manager.loc[(df_manager["conversation_id"] == tweet_id_without_context_list[each]) | 
               (df_manager["referenced_tweet_id"] == str(tweet_id_without_context_list[each]))]

        for each in range(len(df_tweets)):
            if df_tweets.iloc[each]["tweet_id"] not in tweet_id_without_context_list:
                tweet_id_without_context_list.append(df_tweets.iloc[each]["tweet_id"])
                
    if start_len != len(tweet_id_without_context_list):
        check_for_further_tweets()
check_for_further_tweets()

<p>Löschen der nicht kontextbezogenen Tweets und den Index neu setzen um später keine Probleme mit dem Index zu bekommen.</p>

In [8]:
for each in tweet_id_without_context_list:
    df_manager = df_manager.drop(df_manager.loc[df_manager["tweet_id"] == each].index[0])
df_manager = df_manager.reset_index(drop=True)

<p>Löschen und Ermittlung von Tweet_IDs, die keinen Kontext zum Thema haben.</p>

In [9]:
#Text Check up
list_tweet_id = [701784983829143553, 909831998537158657, 909733713302441984, 909730731932368897, 1115628103487954944, 
                1115596390133051393, 1112624842493358080, 1131274228521742336, 1131274228521742336, 1207617572126044160, 
                1265176607905333248, 1273949458808680449, 1285646389771481088, 1285547240447246338, 1285515450584096768, 
                1285497151125151745, 1284925162836709378, 1279724802782216192, 1275482586635255811]
list_tweet_id_remove = []
for tweet in list_tweet_id:
    for each in df_manager.loc[(
        df_manager["tweet_id"] == tweet) | (
        df_manager["conversation_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0]) | (
        df_manager["referenced_tweet_id"] == tweet) | (
        (df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["referenced_tweet_id"].values[0]) & (df_manager["referenced_tweet_id"] != "None")) | (
        df_manager["referenced_tweet_id"] == df_manager.loc[df_manager["tweet_id"] == tweet]["conversation_id"].values[0])]["tweet_id"].values:
        #print(each, "\n")
        if each not in list_tweet_id_remove:
            list_tweet_id_remove.append(each)

<p>Löschen der zuvor ermitteletn Tweets.</p>

In [10]:
for each in list_tweet_id_remove:
    df_manager = df_manager.drop(df_manager.loc[df_manager["tweet_id"] == each].index[0])
df_manager = df_manager.reset_index(drop=True)

<br>
<h3>Löschen von Retweets</h3>
<p>Alle Retweets löschen, da diese in diesem Schritt nicht benötigt werden.</p>

In [11]:
counter = 0
retweet_list = []
for each in range(len(df_manager)):
    if "retweeted" in df_manager.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)

5280


In [12]:
df_manager = df_manager.drop(retweet_list)
df_manager = df_manager.reset_index(drop=True)
print(len(df_manager))

8849


<h3>Löschen von doppelten Tweets</h3>
<p>Alle Tweets, die dopplet im Datensatz enthalten sind, anhand der tweet_id werden gelöscht. Somit werden auch die zuvor aufgespalteten Listen wieder gelöscht</p>

In [13]:
df_manager = df_manager.drop_duplicates(subset=["tweet_id"])
df_manager = df_manager.reset_index(drop=True)

In [14]:
print(len(df_manager))

8706


<h1>Check User</h1>
<p>Überprüfe die User Daten nach Namen, die nichts mit dem Thema Wirecard zu tun haben. Auch Nutzer, die ggf. nur durch Zufall in die Abfrage gekommen sind, z.B. durch den gleichen Namen wie einer der Manager, sollen aus dem Datensatz entnommen werden.<br>Auffällige Namen sind <b>Markus Braun</b> und <b>Jan Marsalek</b>, da es sich um den CEO und den verantwortlichen Manager handelt.</p>

<p>Laden der Daten</p>

In [15]:
df_user = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Raw_File/Manager_DE/user.csv", sep=";")

<h3>Nutzer die keinen Mehrwert bieten werden gelöscht.</h3>

In [16]:
list_not_needed_author_ids = [3365669884, 536335486]
for each in list_not_needed_author_ids:
    df_manager = df_manager.drop(df_manager.loc[df_manager["author_id"] == each].index)
df_manager = df_manager.reset_index(drop=True)

<h1>Hinzufügen weiterer Twitter Datensätze</h1>

<h2>Datensatz mit Abfrage nach Wirecard</h2>

In [17]:
df_wirecard = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Raw_File/Wirecard_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wirecard = df_wirecard.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [18]:
for each in range(len(df_wirecard)):
    df_wirecard.loc[df_wirecard["text"] == 
                      df_wirecard.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$|\r]', '', 
                                                                                       df_wirecard.loc[each]["text"]))))

In [19]:
print(len(df_wirecard))

165934


<h3>Löschen von Retweets</h3>

In [20]:
counter = 0
retweet_list = []
for each in range(len(df_wirecard)):
    if "retweeted" in df_wirecard.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wirecard = df_wirecard.drop(retweet_list)
df_wirecard = df_wirecard.reset_index(drop=True)

54403


In [21]:
len(df_wirecard)

111531

<h3>Löschen von doppelten Einträgen</h3>

In [22]:
df_wirecard = df_wirecard.drop_duplicates(subset=["tweet_id"])
df_wirecard = df_wirecard.reset_index(drop=True)

In [23]:
len(df_wirecard)

111254

<h2>Abfrage nach Chashtag Wirecard</h2>

In [24]:
df_wrcdf = pd.read_csv("/Users/tobias/Dev/FOM/Master_Thesis/data/Raw_File/WRCDF_DE/data.csv", sep=";", 
                            converters={"tweet_type": lambda x: x.strip("[]").replace("'", "").split(", "),
                                        "referenced_tweet_id": lambda x: x.strip("[]").replace("'", "").split(", ")},
                           parse_dates=["created_at"])
df_wrcdf = df_wrcdf.explode(["referenced_tweet_id", "tweet_type"], ignore_index=True)


In [25]:
for each in range(len(df_wrcdf)):
    df_wrcdf.loc[df_wrcdf["text"] == 
                      df_wrcdf.loc[each]["text"], "text"] = re.sub(r'http\S+', '', 
                                                                        (re.sub('@[^\s]+', '',
                                                                                re.sub('[#|$|\r]', '', 
                                                                                       df_wrcdf.loc[each]["text"]))))

In [26]:
print(len(df_wrcdf))

33


<h3>Löschen von Retweets</h3>

In [27]:
counter = 0
retweet_list = []
for each in range(len(df_wrcdf)):
    if "retweeted" in df_wrcdf.iloc[each]["tweet_type"]:
        retweet_list.append(each)
        counter += 1
print(counter)
df_wrcdf = df_wrcdf.drop(retweet_list)
df_wrcdf = df_wrcdf.reset_index(drop=True)

0


<h3>Löschen von doppelten Tweets</h3>

In [28]:
df_wrcdf = df_wrcdf.drop_duplicates(subset=["tweet_id"])
df_wrcdf = df_wrcdf.reset_index(drop=True)

In [29]:
print(len(df_wrcdf))

33


<h1>Zusammenführen der Datensätze</h1>

In [30]:
list_data_frames = [df_manager, df_wirecard, df_wrcdf]
df_de = pd.concat(list_data_frames, ignore_index=True)

In [31]:
print(len(df_manager))
print(len(df_wirecard))
print(len(df_wrcdf))
print(len(df_manager)+len(df_wirecard)+len(df_wrcdf))
print(len(df_de))

8702
111254
33
119989
119989


<h3>Löschen der doppelten Posts</h3>
<p>Dadurch, dass die Abfrage durch mehrere Querys erfolgt ist, befinden sich einige Tweets doppelt in dem Datensatz. Diese werden hier anhand der <b>"tweet_id"</b> gelöscht. </p>

In [32]:
df_de = df_de.drop_duplicates(subset=["tweet_id"])
df_de = df_de.reset_index(drop=True)

<p>Der neue Datensatz wird auf plausibilität geprüft.</p>

In [33]:
print(len(df_de))

114444


<p><b>Fertig</b> mit der Vorverarbeitung der Daten in deutsch. Es ist nun ein Datensatz verfügbar mit allen Tweets für den deutschsprachigen Raum. Es sind keine doppelten Tweets enthalten, solange diese keine gleiche tweet_id enthalten. Auch Nutzer und tweets, die nichts zu dem eigentlichen Thema Wirecard beitragen sind nicht mehr im Datensatz enthalten. Vereinzelt können natürlich unnütze Tweets vorhanden sein, aber die Nutzer mit vielen Interaktionen sind überprüft und auch die Begriffe, die besonders oft vorkommen sind überprüft worden, ob diese etwas mit dem Thema zu tun haben.<br>Jetzt muss das gleich noch mit den Englischen tweets geschehen und dann müssen Datensätze zur Überprüfung ausgegeben werden.<br> Um den zufälligen Datensatz zu erstellen sollte ich noch einen zufallszahlengenerator nutzen, sodass ich möglichst zufällige Tweets zum testen erhalte. Hier scheint die Pandas Funktion <b>"sample"</b> gut geeignet zu sein. </p>

In [34]:
df_de.sample(n = 20)

,conversation_id,author_id,created_at,tweet_id,lang,retweet_count,reply_count,like_count,quote_count,source,tweet_type,referenced_tweet_id,text
93926,1288856739190906882,236077053,2020-07-30 15:19:27+00:00,1288856739190906882,de,1,2,1,2,Twitter for iPhone,None,None,Warum im Wirecard-Skandal ein Untersuchungsaus...
100981,1285775134477950981,27800574,2020-07-22 03:14:15+00:00,1285775134477950981,de,1,0,1,0,Twitter for Android,None,None,gewerbe steuer einnahmen aschheim dank wirecar...
16092,822048375038181376,2807967368,2017-01-19 11:49:40+00:00,822048375038181376,de,0,0,0,0,IFTTT,None,None,Finanznachrichten: Goldman hebt Ziel für Wirec...
65092,1263499578516660226,1159698800790953984,2020-05-21 15:59:08+00:00,1263499578516660226,de,2,0,3,0,Twitter for iPhone,None,None,So wie es aussieht strebt der Bund eine Beteil...
7956,1283428752605810694,123781907,2020-07-16 04:27:59+00:00,1283619363820642304,de,0,0,0,0,Twitter for Android,replied_to,1283428752605810694,Was wusste unser BND oder gar der Verfassungs...
81320,1275094905832996865,17803524,2020-06-22 15:54:50+00:00,1275094905832996865,de,1,1,3,1,SZ.de Twitterbot - Topthemen,None,None,"Wirecard räumt ein, dass es einen Großteil des..."
105130,1284010202027761664,808400627084705792,2020-07-17 06:21:02+00:00,1284010202027761664,de,8,1,56,0,Twitter for iPhone,None,None,Wir stehen erst am Anfang: Noch immer sind vie...
94415,1288730693963583488,717955884,2020-07-30 06:58:35+00:00,1288730693963583488,de,6,4,8,0,Twitter Web App,None,None,Olaf Scholz präsentierte sich in der vierstünd...
53804,1199977245600886784,37382601,2019-11-28 09:04:04+00:00,1199977245600886784,de,0,0,0,0,Josef Chladek,None,None,"wikifolio whispers a.m.: Wirecard, Vodafone, V..."
26294,1031850922719240196,21864513,2018-08-21 10:30:05+00:00,1031850922719240196,de,0,0,0,0,Blog2Social APP,None,None,Die kleine Presseschau vom 21. August 2018 mi...


<h2>Finalen Datensatz als CSV ausgeben.</h2>

In [35]:
df_de.to_csv('Dataset_Wirecard.csv', sep=';', index=False)

In [36]:
df_validation_sentiment.head()

NameError: name 'df_validation_sentiment' is not defined

<h2>Erstellen der CSV zur Überprüfung der Sentiment Ergebnisse</h2>

In [37]:
df_de.sample(n=10)[["tweet_id", "text"]]

,tweet_id,text
97992,1286659544941236225,Aktien in diesem ArtikelBERLIN (dpa-AFX) - Im ...
54283,1197083936670863360,Aktienkurs sinkt: Jahresabschluss von Wirecard...
99248,1286230482439606272,Mein Name ist Merkel. Ich weiß von nichts!
16222,819826381559386112,🔍 Wirecard Communication Services GmbH sucht S...
75739,1276101375429095424,"Weil es vielleicht noch Leute gibt, die hof..."
76526,1276077813674553345,Insolvenz: Wirecard meldet Insolvenz an
41028,1107320978407936003,Wirecard: Das Vertrauen entzogen!
61711,1255374137868062720,Die kommen aus dem Loch nicht raus. Womögl...
18383,863026030830866432,Trotz Attacken von Leerverkäufern - die Wireca...
68100,1257718289167458305,Wirecard übernimmt Zahlungsabwicklung für Lief...


In [38]:
df_sentiment_validation = df_de.sample(n = 1500)[["tweet_id", "text"]]
batch = 14
counter = 0 
for batch in range(0,15):
    start = 100*batch
    end = 100*batch+100
    csv_name = 'de_validation_{count}.csv'.format(count=batch)
    df_sentiment_validation.iloc[start:end].to_csv(csv_name, sep=';', index=False)
    print("Start bei {start} und ende bei {ende}. Name der Datei: {csv}".format(start=start, ende=end, csv=csv_name))

Start bei 0 und ende bei 100. Name der Datei: de_validation_0.csv
Start bei 100 und ende bei 200. Name der Datei: de_validation_1.csv
Start bei 200 und ende bei 300. Name der Datei: de_validation_2.csv
Start bei 300 und ende bei 400. Name der Datei: de_validation_3.csv
Start bei 400 und ende bei 500. Name der Datei: de_validation_4.csv
Start bei 500 und ende bei 600. Name der Datei: de_validation_5.csv
Start bei 600 und ende bei 700. Name der Datei: de_validation_6.csv
Start bei 700 und ende bei 800. Name der Datei: de_validation_7.csv
Start bei 800 und ende bei 900. Name der Datei: de_validation_8.csv
Start bei 900 und ende bei 1000. Name der Datei: de_validation_9.csv
Start bei 1000 und ende bei 1100. Name der Datei: de_validation_10.csv
Start bei 1100 und ende bei 1200. Name der Datei: de_validation_11.csv
Start bei 1200 und ende bei 1300. Name der Datei: de_validation_12.csv
Start bei 1300 und ende bei 1400. Name der Datei: de_validation_13.csv
Start bei 1400 und ende bei 1500. Nam